# Redes recurrentes

Ejemplo

In [ ]:
import numpy as np

TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

In [ ]:
print(questions[0])
print(expected[0])

In [ ]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector
from tensorflow.keras.models import Model


i = Input((MAXLEN, len(chars)))
d = LSTM(128, return_sequences=False)(i)
d = RepeatVector(DIGITS+1)(d)
d = LSTM(128, return_sequences=True)(d)
d = Dense(len(chars), activation='softmax')(d)

model = Model(i, d)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
epochs = 15 #30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)

## IMDB 

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [ ]:
import os
from sklearn.utils import shuffle
import numpy as np

def load_imdb_ds(path, shuffle_ds=True, random_state=None):
    def load_text(path_sel):
        xa = []
        for f in os.listdir(path_sel):
            f = open(path_sel + os.sep + f, 'r')
            xa.append(next(f))
        return xa
    x = load_text(path + os.sep + 'pos')
    y = [1] * len(x)
    xn = load_text(path + os.sep + 'neg')
    x.extend(xn)
    y.extend([0] * len(xn))
    if shuffle_ds:
        shuffle(x, y, random_state=random_state)
    return x, y

x_train_text, y_train = load_imdb_ds('aclImdb/train', random_state=42)
x_test_text, y_test = load_imdb_ds('aclImdb/test', random_state=42)

In [ ]:
x_train_text[:10]

In [ ]:
!pip install bs4
!pip install tqdm

from collections import Counter, deque
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re 

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text


import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#Baja los stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]


def tokenizer_simple(text):
    return [w for w in re.split('\s+', text.strip()) \
            if re.match('[a-zA-Z]+', w)]


def process_corpus(data, words_id=None, min_reps=5, tokenizer=tokenizer_simple):
    corpus = []
    for text in tqdm(data):
        corpus.append(tokenizer(preprocessor(text)))
        
    if words_id is None:
        #Cuenta palabras en el corpus
        words = Counter()

        for s in corpus:
            for w in s:
                words[w] += 1

        #Elimina palabras con menos de 5 repeticiones
        words_id = {}
        id_next = 1
        for w, c in words.items():
            if c >= min_reps:
                words_id[w] = id_next
                id_next += 1

    id_words = { v:k for k, v in words_id.items()}
    corpus_id = [[words_id[w] for w in s if w in words_id] for s in corpus]
    return corpus_id, words_id, id_words

In [ ]:
x_train, words_id, id_words = process_corpus(x_train_text)
x_test, _, _ = process_corpus(x_test_text, words_id=words_id)
y_train = np.expand_dims(np.asarray(y_train), axis=-1)
y_test = np.expand_dims(np.asarray(y_test), axis=-1)

In [ ]:
print(x_train[:5])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAXLEN = 50

x_train = pad_sequences(x_train, maxlen=MAXLEN)
x_test = pad_sequences(x_test, maxlen=MAXLEN)

In [ ]:
print(x_train.shape)
print(x_train)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Embedding
from tensorflow.keras.models import Model

i = Input((None,))
d = Embedding(len(words_id) + 1, 300, mask_zero=True)(i)
d = Dropout(0.5)(d)
d = LSTM(128, return_sequences=True)(d)
d = Dropout(0.5)(d)
d = LSTM(128, return_sequences=False)(d)
d = Dropout(0.5)(d)
d = Dense(1, activation='sigmoid')(d)

model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

## Embeddings preentrenados

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
from matplotlib import pyplot as plt
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec

glove2word2vec(glove_input_file='glove.6B.300d.txt', word2vec_output_file='vectors.txt')
glove = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False)

In [ ]:
from tensorflow.keras import backend as K

emb = Embedding(len(words_id) + 1, 300, mask_zero=True, trainable=False)
i = Input((None,))
d = emb(i)
d = Dropout(0.5)(d)
d = LSTM(128, return_sequences=True)(d)
d = Dropout(0.5)(d)
d = LSTM(128, return_sequences=False)(d)
d = Dropout(0.5)(d)
d = Dense(1, activation='sigmoid')(d)

model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])

In [ ]:
base_emb = np.zeros(emb.embeddings.shape)

for w, i in words_id.items():
    if w in glove:
        base_emb[i, :] = (glove[w] / np.sum(glove[w]))

K.set_value(emb.embeddings, base_emb)
emb = None

In [ ]:
model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

## Generacion de texto

In [ ]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
i = Input((maxlen, len(chars)))
d = LSTM(128)(i)
d = Dense(len(chars), activation='softmax')(d)

model = Model(i, d)

optimizer = 'adam'#RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(60):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

# Attention

In [ ]:
from tensorflow.keras.layers import Attention, Add, GlobalAveragePooling1D, Dropout, Lambda, Embedding, Input, Dense
import tensorflow as tf
from tensorflow.keras.models import Model


i = Input((None,))
e = Embedding(len(words_id) + 1, 60, mask_zero=True, name='base_emb')(i)
len_s = Lambda(lambda x: tf.expand_dims(tf.range(start=0, limit=MAXLEN, delta=1), axis=0))(i)
pos_e = Embedding(MAXLEN, 60, mask_zero=True, name='base_pos')(len_s)

sum = Add()([e, pos_e])
dq = Dense(60)(sum)
dk = Dense(60)(sum)

mask = Lambda(lambda x: x != 0)(i)

att = Attention()([dq, dk], mask=[mask, mask])
attd = Dropout(0.1)(att)

d = GlobalAveragePooling1D()(attd)
d = Dropout(0.1)(d)
d = Dense(100)(d)
d = Dropout(0.1)(d)
d = Dense(1, activation='sigmoid')(d)


model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])


model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
from tensorflow.keras.layers import Attention, Add, GlobalAveragePooling1D, Dropout
import tensorflow as tf


i = Input((None,))
e = Embedding(len(words_id) + 1, 60, mask_zero=True, name='base_emb')(i)

dq = Dense(60)(e)
dk = Dense(60)(e)

att = Attention()([dq, dk])
attd = Dropout(0.1)(att)

d = GlobalAveragePooling1D()(attd)
d = Dropout(0.1)(d)
d = Dense(100)(d)
d = Dropout(0.1)(d)
d = Dense(1, activation='sigmoid')(d)


model = Model(i, d)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

In [ ]:
x_exam = ['The movie was excelent. It is probably the best movie ever', 'The movie was not good']
x_exam_v, _, _ = process_corpus(x_exam, words_id)
x_exam_v = pad_sequences(x_exam_v, maxlen=MAXLEN)

In [ ]:
print(model.predict(x_exam_v))

In [ ]:
x_exam_v

In [ ]:
model_att = Model(i, [dq, dk])

vq, vk = model_att(x_exam_v)

In [ ]:
att_sal = tf.nn.softmax(tf.matmul(vq, vk, transpose_b=True))
print(att_sal[0,-10:, -10:])
print(att_sal[0,-5:, -5:])

In [ ]:
import matplotlib.pyplot as plt


plt.imshow(att_sal[0,...])
plt.show()
plt.imshow(att_sal[0, -10:, -10:])
plt.show()

In [ ]:
plt.imshow(att_sal[1,...])
plt.show()
plt.imshow(att_sal[1, -5:, -5:])
plt.show()

In [ ]:
print(x_test_text[0])
print(model.predict(np.expand_dims(x_test[0, :], axis=0)))
vq, vk = model_att(np.expand_dims(x_test[0, :], axis=0))
att_sal = tf.nn.softmax(tf.matmul(vq, vk, transpose_b=True))
plt.imshow(att_sal[0,...])
plt.show()